In [1]:
import yfinance as yf
import pandas as pd
from numpy import where
from pandas import concat

In [2]:
# write function to download table from yfinance https://pypi.org/project/yfinance/
# pip install yfinance (to pip install)
# pip install yfinance --upgrade (to upgrade, latest version 0.1.62 as of 2021-07-09)

def get_tables(stock_list):
    """
    Function takes list object and pickles the dataframes for use later
    Inputs: stock_list (python list object of lower case stock ticker symbols)
    Ouputs: stores .pkl object in data folder
    """
    for stock in stock_list:
        stock_df = yf.Ticker(stock.upper()).history(period="max")
        stock_df.to_pickle(f'./data/{stock.lower()}_df.pkl')
        
# list of common stock ticker symbols
stock_list = ['aapl', 'ibm', 'tsla', 'vgt']

# call function to pickle dataframes after downloads
get_tables(stock_list)

In [ ]:
# write function to transform dataframe and split for train/test dataframe sets

def data(stock, days_ahead):
    """
    Inputs: stock, string of stock symbol like 'aapl' or 'AAPL'
            days_ahead, int days prediction ahead, 1 for 1 day ahead, 2 for 2 days ahead, etc...
    Output: X_train, X_test, y_train, y_test (for modeling), stock_df (DataFrame)
    """
    
    # load stock data pickle file from data folder
    stock_df = pd.read_pickle(f'./data/{stock.lower()}_df.pkl')
    
    # some open values are 0.00, set as their close value, otherwise leave as open value
    # this is done so we don't divide by zero, see next line for 'oc' calculation
    stock_df['Open'] = where(stock_df['Open'] == 0, stock_df['Close'], stock_df['Open'])
    
    # calculate daily open close % difference
    stock_df['oc'] = (stock_df.Close - stock_df.Open) / stock_df.Open
    
    # calculate daily high low % difference
    stock_df['hl'] = (stock_df.High - stock_df.Low) / stock_df.Low
    
    # % change from pervious day Close
    stock_df['prev_change'] = stock_df['Close'].pct_change()
    
    # 13 day rolling moving standard deviation of prev_change
    stock_df['13std'] = stock_df.prev_change.rolling(13).std()
    
    # 13 day rolling moving of prev_change
    stock_df['13mva'] = stock_df.prev_change.rolling(13).mean()
    
    # Direction (target feature column)
    stock_df['direction'] = where(stock_df['prev_change'].shift(-days_ahead) > stock_df['prev_change'], 1, -1)
    
    # Drop nulls generated from rolling columns
    stockdf.dropna(axis=0, inplace=True)
    
    # 
    